<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/intervviewZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install git+https://github.com/openai/whisper.git
!pip install -U git+https://github.com/jianfch/stable-ts.git

  Cloning https://github.com/jianfch/stable-ts.git to /tmp/pip-req-build-hkvrk9ga
  Running command git clone --filter=blob:none --quiet https://github.com/jianfch/stable-ts.git /tmp/pip-req-build-hkvrk9ga
  Resolved https://github.com/jianfch/stable-ts.git to commit 7dab1714c5915e3efe070ae697ade1d04f5c8c42
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 40.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
  Created wheel for stable-ts: filename=stable_ts-2.17.5-py3-none-any.whl size=136044 sha256=aac8621d2d695911b7051cb1e43804b3cbc6da95adb96c1d3c85531cb8da1311
  Stored in directory: /tmp/pip-ephem-wheel-cache-ufcuhpw_/wheels/5a/48/64/a463d57ac05105e1692e3649ca76cea98a8867262d

In [2]:
!pip install moviepy

In [3]:
!pip install ffmpeg-python

In [4]:
#import whisper
import stable_whisper
import ffmpeg
import numpy as np
import json, re, os
import glob
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from moviepy.editor import VideoFileClip
from moviepy.editor import AudioFileClip

  if event.key is 'enter':



In [5]:
!cp ./drive/MyDrive/IZ-2218確認４/*.* ./

In [6]:
def is_webm(path):
    with open('./' + path) as f:
        for _line in f:
            if _line.rstrip()[-5:] == '.webm':
                return True
    return False

In [7]:
def conv_webm(input_file):
    #!ffmpeg -allowed_extensions ALL -f hls -i ./drive/MyDrive/IZ-2218確認３/ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.m3u8 -c:v copy -y ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm
    webm_tmp = "./video_tmp.webm"
    try:
        (
            ffmpeg
            .input('./' + input_file, allowed_extensions='ALL')
            .output(webm_tmp, vcodec="copy")
            .overwrite_output()
            .run(capture_stderr=True, capture_stdout=True)
        )
        _clip = VideoFileClip(webm_tmp)
    except ffmpeg.Error as e:
        #print(f"{e.stderr.decode()}")
        #print(f"標準出力: {e.stdout.decode()}")
        _clip = None
    return _clip

In [8]:
def get_video_list(path):
    video_path = f'{path}*_video.m3u8'
    video_paths = glob.glob(video_path)
    video_paths.sort()
    return video_paths
def get_audio_list(path):
    audio_path = f'{path}*_audio.m3u8'
    audio_paths = glob.glob(audio_path)
    audio_paths.sort()
    return audio_paths

In [9]:
path = './'
video_paths = get_video_list(path)
audio_paths = get_audio_list(path)

In [10]:
video = []
for video_path in video_paths:
    _video = conv_webm(video_path) if is_webm(video_path) else VideoFileClip(video_path)
    video.append(_video)
print(video)

[<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7fb3d4b988e0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7fb50cd0e530>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7fb50cc6a6b0>]


In [11]:
audio = []
wav_paths = []
for audio_path in audio_paths:
    wav_path = '.'.join(audio_path.split('.')[:-1]) + '.wav'
    audio_tmp = AudioFileClip(audio_path)
    audio_tmp.write_audiofile(wav_path, ffmpeg_params=["-ac", "1"])
    audio.append(audio_tmp)
    wav_paths.append(wav_path)
print(audio)

MoviePy - Writing audio in ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_186868730__uid_e_audio.wav


MoviePy - Done.
MoviePy - Writing audio in ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_801600513__uid_e_audio.wav


MoviePy - Done.
MoviePy - Writing audio in ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_846732376__uid_e_audio.wav


MoviePy - Done.
[<moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7fb3d4b98970>, <moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7fb50cd0db70>, <moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7fb3d4b9be80>]


In [12]:
#model = whisper.load_model("large")
model = stable_whisper.load_model('large')
stable_whisper.modify_model(model)

100%|██████████████████████████████████████| 2.88G/2.88G [00:23<00:00, 129MiB/s]
  checkpoint = torch.load(fp, map_location=device)



In [13]:
asr_result = []
for wav_patn in wav_paths:
    print(f'\n---- {wav_patn} ----')
    _result = model.transcribe(wav_patn, verbose=True, fp16=False, language="ja", word_timestamps=True)
    asr_result.append(_result)


---- ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_186868730__uid_e_audio.wav ----
[00:03.340 --> 00:11.540] "聞こえますか"
-[00:03.340] -> [00:07.000] "聞"
-[00:10.620] -> [00:10.840] "こ"
-[00:10.860] -> [00:11.060] "え"
-[00:11.060] -> [00:11.300] "ます"
-[00:11.300] -> [00:11.540] "か"

[00:11.720 --> 00:16.460] "高橋さんはどうでしょう"
-[00:11.720] -> [00:14.480] "高"
-[00:14.940] -> [00:15.040] "橋"
-[00:15.040] -> [00:15.540] "さん"
-[00:15.700] -> [00:15.880] "は"
-[00:15.880] -> [00:16.060] "どう"
-[00:16.060] -> [00:16.460] "でしょう"

[00:16.780 --> 00:17.980] "聞こえますか"
-[00:16.780] -> [00:17.420] "聞"
-[00:17.460] -> [00:17.560] "こ"
-[00:17.560] -> [00:17.600] "え"
-[00:17.600] -> [00:17.780] "ます"
-[00:17.780] -> [00:17.980] "か"

[00:21.640 --> 00:26.460] "高橋さんなんか喋って"
-[00:21.640] -> [00:21.780] "高"
-[00:21.780] -> [00:25.460] "橋"
-[00:25.460] -> [00:25.700] "さん"
-[00:25.800] -> [00:26.000] "なんか"
-[00:26.000] -> [00:26.300] "喋"
-[00:26.300] -> [00:26.460] "って"

[00:35.040 --> 00:36.400] "聞こえてないかも"
-

In [23]:
dir(asr_result[0])

['__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_forced_order',
 '_merge_segments',
 '_nonspeech_sections',
 '_regroup_history',
 '_split_segments',
 '_standardize_result',
 'add_segments',
 'adjust_by_result',
 'adjust_by_silence',
 'adjust_gaps',
 'all_tokens',
 'all_words',
 'all_words_by_lock',
 'all_words_or_segments',
 'apply_min_dur',
 'clamp_max',
 'convert_to_segment_level',
 'duration',
 'fill_in_gaps',
 'find',
 'force_order',
 'get_content_by_time',
 'get_gap_indices',
 'get_gaps',
 'get_locked_indices',
 'get_punctuation_indices',
 'has_words',
 'language',
 'lock',
 'merge_all_segments',
 'merge_by_gap',
 'merg

In [33]:
#asr_result[0].segments
asr_result[0].all_words_or_segments()
#stab_segments = stable_whisper.stabilize_timestamps(asr_result[0].segments, top_focus=True)


[WordTiming(start=6.4, end=7.0, word="聞"),
 WordTiming(start=10.62, end=10.84, word="こ"),
 WordTiming(start=10.86, end=11.06, word="え"),
 WordTiming(start=11.06, end=11.3, word="ます"),
 WordTiming(start=11.3, end=11.54, word="か"),
 WordTiming(start=13.23, end=14.48, word="高"),
 WordTiming(start=14.94, end=15.04, word="橋"),
 WordTiming(start=15.04, end=15.54, word="さん"),
 WordTiming(start=15.7, end=15.88, word="は"),
 WordTiming(start=15.88, end=16.06, word="どう"),
 WordTiming(start=16.06, end=16.46, word="でしょう"),
 WordTiming(start=16.92, end=17.42, word="聞"),
 WordTiming(start=17.46, end=17.56, word="こ"),
 WordTiming(start=17.56, end=17.6, word="え"),
 WordTiming(start=17.6, end=17.78, word="ます"),
 WordTiming(start=17.78, end=17.98, word="か"),
 WordTiming(start=21.64, end=21.78, word="高"),
 WordTiming(start=21.78, end=25.46, word="橋"),
 WordTiming(start=25.46, end=25.7, word="さん"),
 WordTiming(start=25.8, end=26.0, word="なんか"),
 WordTiming(start=26.0, end=26.3, word="喋"),
 WordTiming(start

In [ ]:
whisper_result = {}
whisper_segment = []
for asr in asr_result['segments']:
    if asr['end'] - asr['start'] >= 0.6 and len(asr['text']) >= 2:
        asr['text'] += '。'
        whisper_segment.append(asr)
whisper_result['segments'] = whisper_segment

In [ ]:
whisper_result

In [ ]:
#speaker_count = 2
speaker_count = 3

import torch
pipeline.to(torch.device("cuda"))
diarization_result = pipeline(audio_path, num_speakers=speaker_count)
#diarization_result = pipeline(audio_path, min_speakers=2, max_speakers=5)

In [ ]:
diarization_result

In [ ]:
#final_result = diarize_text(asr_result, diarization_result)
final_result = diarize_text(whisper_result, diarization_result)

In [ ]:
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
INT_COUNT = []
for seg, spk, sent in final_result:
  if spk is None:
    spk = INTERVIEW[-1]['spk']
  count = len(prog.findall(sent))
  spk_no = int(spk.split('_')[-1])
  if count > 0: print(str(sent))
  _seg = {'start': round(seg.start, 2), 'end': round(seg.end, 2), 'text': sent, 'count': count, 'spk': spk, 'spk_no': spk_no}
  INTERVIEW.append(_seg)
  INT_COUNT.append(int(spk[-1]))

speaker_count = max(INT_COUNT) + 1

speek = [0] * speaker_count
for INT in INTERVIEW:
    speek[INT["spk_no"]] += INT["count"]
inteviewer_index = np.argmax(speek)
monitor_index = np.argmin(speek)

print(speek)
print(f'インタービューア ： SPEAKER_0{inteviewer_index}')
print(f'モニター　　　　 ： SPEAKER_0{monitor_index}')

In [ ]:
for seg, spk, sent in final_result:
    speeker_name = spk
    #speeker_name =   'モニタ　　　　'
    #if spk == 'SPEAKER_0' + str(inteviewer_index):
    #  speeker_name = 'インタビューア'
    line = f'{seg.start:.2f}s~{seg.end:.2f}s {speeker_name} {sent}'
    print(line)

In [ ]:
InterReview_json = {"movie_name": video_name}
speak_json = []
for seg, spk, sent in final_result:
    _speak = {"start": f'{seg.start:.2f}s', "end": f'{seg.end:.2f}s', 'speaker': spk, 'speak': sent}
    speak_json.append(_speak)
InterReview_json['InterReview'] = speak_json

json_path = '.'.join(video_path.split('.')[:-1]) + '.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(InterReview_json, f, indent=4, ensure_ascii=False)


In [ ]:
!ls -alh ./

In [ ]:
!cp ./InterviewZero_0_audio_mask.json ./drive/MyDrive

In [ ]:
!cp ./InterviewZero_1_audio_mask.json ./drive/MyDrive

In [ ]:
!cp ./InterviewZero_2_audio_mask.json ./drive/MyDrive